In [3]:
import time
from datetime import datetime as dt
from biscoint_api_python import Biscoint
from playsound import playsound
import threading
from utils import percent, btcToTrade
from configs import logging
from configs import API_KEY, API_SECRET,AMOUNT, MIN_PERCENT_REQUIRED,BASE_URL, BRL_AMOUNT_TRADE, UPDATE_TICK_RATE

In [4]:
bsc = Biscoint(API_KEY, API_SECRET)
endpoints_meta = bsc.get_meta()
rate_limit_offer = endpoints_meta['endpoints']['offer']['post']['rateLimit']
sleep_time_offers = ((rate_limit_offer["windowMs"] / rate_limit_offer["maxRequests"]) / 1000)*2
initial_balance = bsc.get_balance()
last_balance = initial_balance
ticker = bsc.get_ticker()
amount_btc_to_trade = btcToTrade(BRL_AMOUNT_TRADE,ticker['askQuoteAmountRef'],ticker['bidBaseAmountRef'])

In [5]:
def async_offer(op:str,amount:str,isQuote:bool):
    response = None
    try:
        response = bsc.get_offer(op,amount,isQuote) 
        request_orders[op] = response
    except Exception as e:
        print(response)
        print("Erro ao rodar thread",e)

def updateTick(cycle_count):
    if cycle_count % UPDATE_TICK_RATE == 0:
        try:
            ticker = bsc.get_ticker()
            amount_btc_to_trade = btcToTrade(BRL_AMOUNT_TRADE,ticker['askQuoteAmountRef'],ticker['bidBaseAmountRef'])
        except Exception as e:
            logging.error(f"Error on updating tick {e}")

# if spread is high, sleep, else speed up checks
def waitForNextCycle(calculated_percent):
    if calculated_percent < -0.3:
        time.sleep(sleep_time_offers)
    else:
        time.sleep(2)

In [7]:
# Arbitrage Cycle
cycle_count = 1
percent_record = -1
request_orders = {'buy':None,'sell':None}
while True:
    try:
        start_time = dt.now()
        buy = bsc.get_offer(op='buy',amount=str(amount_btc_to_trade),isQuote=False)
        sell = bsc.get_offer(op='sell',amount=str(amount_btc_to_trade),isQuote=False)

        # thread_buy = threading.Thread(target=async_offer,args=('buy',str(amount_btc_to_trade),False))
        # thread_sell = threading.Thread(target=async_offer,args=('sell',str(amount_btc_to_trade),False))
        # thread_buy.start()
        # time.sleep(0.1)
        # thread_sell.start()
        # thread_buy.join()
        # thread_sell.join()


    # Calculate if arbitrage is possible
        calculated_percent = percent(buy['efPrice'],sell['efPrice'])
        logging.info(f"Percent is {calculated_percent} | Cycle {cycle_count}")

        if calculated_percent > percent_record:
            logging.info(f"Percent Record Reached!! : {calculated_percent}")
            percent_record = calculated_percent
        # If Arbitrage is possible, confirm offers
        if calculated_percent >= MIN_PERCENT_REQUIRED:
            logging.info(f"Arbitrage oportunity: buy:{buy['efPrice']}   sell:{sell['efPrice']}")
            playsound('beep.wav')
            #Execute orders
            executed_buy = bsc.confirm_offer(buy['offerId'])
            executed_sell = bsc.confirm_offer(sell['offerId'])
            logging.info(executed_buy)
            logging.info(executed_sell)

            last_balance = bsc.get_balance()
            logging.info(f"New Balance is: {last_balance}")
            

        end_time = dt.now()
        seconds_elapsed = (end_time - start_time).total_seconds()
        logging.debug(f"Cycle took {seconds_elapsed} seconds")
        cycle_count +=1

        updateTick(cycle_count)
        waitForNextCycle(calculated_percent)
    except Exception as e:
        logging.error(e)
        print(e)

Percent is -0.434 | Cycle 1
Percent Record Reached!! : -0.434
Percent is -0.434 | Cycle 2


KeyboardInterrupt: 

In [ ]:
#thread_buy = threading.Thread(target=offerFunc,args=('buy',AMOUNT,False,buy))
#thread_sell = threading.Thread(target=offerFunc,args=('sell',AMOUNT,False,sell))
#thread_buy.start()
#thread_sell.start()
#thread_buy.join()
#thread_sell.join()
